In [2]:
import ccxt
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate, Dropout, multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Конфигурация модели
TIMEFRAMES = ['1m', '5m', '15m', '1h', '4h', '1d', '1w']
LOOKBACK_PERIODS = {
    '1m': 60,    
    '5m': 12,     
    '15m': 4,     
    '1h': 24,     
    '4h': 168,    
    '1d': 30,     
    '1w': 12      
}
FEATURES = ['open', 'high', 'low', 'close', 'volume']
NUM_FEATURES = len(FEATURES)

def fetch_multitimeframe_data(symbol='BTC/USDT', exchange_name='binance'):
    exchange = getattr(ccxt, exchange_name)({
        'enableRateLimit': True,
        'options': {'defaultType': 'spot'}
    })
    
    exchange.load_markets()
    data = {}
    
    for tf in TIMEFRAMES:
        try:
            print(f"Загрузка {tf} данных...")
            ohlcv = exchange.fetch_ohlcv(symbol, tf, limit=1000)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            data[tf] = df.dropna()
            time.sleep(exchange.rateLimit / 1000)
        except Exception as e:
            print(f"Ошибка при загрузке {tf}: {str(e)}")
            continue
    
    return data

def align_datasets(raw_data):
    min_samples = min([len(raw_data[tf]) - LOOKBACK_PERIODS[tf] for tf in TIMEFRAMES if tf in raw_data]) - 1
    
    aligned_data = {}
    scalers = {}
    
    for tf in TIMEFRAMES:
        if tf not in raw_data or len(raw_data[tf]) < LOOKBACK_PERIODS[tf] + min_samples:
            continue
            
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(raw_data[tf][FEATURES])
        scalers[tf] = scaler
        
        X = []
        start_idx = len(scaled_data) - min_samples - LOOKBACK_PERIODS[tf]
        for i in range(start_idx, len(scaled_data) - LOOKBACK_PERIODS[tf]):
            X.append(scaled_data[i:i+LOOKBACK_PERIODS[tf]])
            
        aligned_data[tf] = np.array(X)
    
    return aligned_data, scalers, min_samples

def prepare_training_data(datasets, min_samples):
    X_train = {tf: data[:min_samples-1] for tf, data in datasets.items()}
    
    y_price = datasets['1m'][1:min_samples, -1, 3]
    y_signal = (datasets['1m'][1:min_samples, -1, 3] > datasets['1m'][:min_samples-1, -1, 3]).astype(int)
    
    return X_train, {'price_prediction': y_price, 'signal': y_signal}

def create_model(input_shapes):
    inputs = []
    branches = []
    
    for tf, shape in input_shapes.items():
        inp = Input(shape=shape, name=f'input_{tf}')
        x = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01))(inp)
        x = Dropout(0.3)(x)
        x = LSTM(32, kernel_regularizer=l2(0.01))(x)
        branches.append(x)
        inputs.append(inp)
    
    merged = concatenate(branches)
    
    attention = Dense(len(branches)*32, activation='softmax')(merged)
    attended = multiply([merged, attention])
    
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(attended)
    x = Dropout(0.4)(x)
    
    reg_output = Dense(1, name='price_prediction')(x)
    class_output = Dense(1, activation='sigmoid', name='signal')(x)
    
    model = Model(inputs=inputs, outputs=[reg_output, class_output])
    model.compile(
        optimizer=Adam(0.001),
        loss={'price_prediction': 'mse', 'signal': 'binary_crossentropy'},
        metrics={'price_prediction': ['mae'], 'signal': ['accuracy']}
    )
    return model

def generate_signal(current_price, predicted_price, threshold=0.001):
    price_change = (predicted_price - current_price) / current_price
    if price_change > threshold:
        return 'buy'
    elif price_change < -threshold:
        return 'sell'
    else:
        return 'hold'

if __name__ == "__main__":
    raw_data = fetch_multitimeframe_data()
    datasets, scalers, min_samples = align_datasets(raw_data)

    if not datasets:
        raise ValueError("Недостаточно данных для обучения")

    print(f"Используется {min_samples-1} образцов для обучения")

    X_train, y_train = prepare_training_data(datasets, min_samples)

    input_shapes = {tf: (LOOKBACK_PERIODS[tf], NUM_FEATURES) for tf in datasets}
    model = create_model(input_shapes)

    # ✅ Переименование ключей для соответствия входам модели
    X_train_named = {f'input_{tf}': data for tf, data in X_train.items()}

    # Обучение модели
    history = model.fit(
        X_train_named,
        y_train,
        epochs=10,
        batch_size=64,
        validation_split=0.2,
        verbose=1
    )

    print("Прогнозирование...")

    # ✅ Подготовка текущих данных (используем последние LOOKBACK_PERIOD samples)
    current_data = {}
    for tf in datasets:
        last_seq = datasets[tf][-1]  # последний фрагмент размером LOOKBACK_PERIOD
        current_data[f'input_{tf}'] = np.expand_dims(last_seq, axis=0)  # добавляем размерность батча

    price_pred, signal_prob = model.predict(current_data)

    signal = generate_signal(
        current_price=raw_data['1m'].iloc[-1]['close'],
        predicted_price=price_pred[-1][0],
        threshold=0.002
    )

    print("\nРезультаты:")
    print(f"Текущая цена: {raw_data['1m'].iloc[-1]['close']}")
    print(f"Прогнозируемая цена: {price_pred[-1][0]}")
    print(f"Вероятность сигнала: {signal_prob[-1][0]:.2f}")
    print(f"Рекомендуемое действие: {signal.upper()}")

Загрузка 1m данных...
Загрузка 5m данных...
Загрузка 15m данных...
Загрузка 1h данных...
Загрузка 4h данных...
Загрузка 1d данных...
Загрузка 1w данных...
Используется 393 образцов для обучения
Epoch 1/10
5/5 [==============================] - 11s 644ms/step - loss: 10.3596 - price_prediction_loss: 1.6841 - signal_loss: 0.6931 - price_prediction_mae: 1.1076 - signal_accuracy: 0.5414 - val_loss: 11.4862 - val_price_prediction_loss: 3.2540 - val_signal_loss: 0.6931 - val_price_prediction_mae: 1.7747 - val_signal_accuracy: 0.5063
Epoch 2/10
5/5 [==============================] - 1s 113ms/step - loss: 9.6115 - price_prediction_loss: 1.6530 - signal_loss: 0.6928 - price_prediction_mae: 1.1053 - signal_accuracy: 0.5223 - val_loss: 10.6866 - val_price_prediction_loss: 3.1410 - val_signal_loss: 0.6931 - val_price_prediction_mae: 1.7426 - val_signal_accuracy: 0.5063
Epoch 3/10
5/5 [==============================] - 1s 112ms/step - loss: 8.9074 - price_prediction_loss: 1.6156 - signal_loss: 0.69

In [4]:
dsadaa =2